# ODIBI Getting Started - Complete Walkthrough

**Welcome to ODIBI!** This notebook will teach you everything you need to know to build data pipelines.

## What You'll Learn:

1. ✅ Basic pipeline (read → write)
2. ✅ Transform functions (custom logic)
3. ✅ SQL transforms
4. ✅ Multi-source pipelines (joins)
5. ✅ Debugging techniques
6. ✅ Error handling

## Prerequisites:

- ODIBI installed: `pip install -e d:/odibi`
- This notebook in `examples/getting_started/`
- Sample data in `data/` folder

Let's get started! 🚀

---

## Setup

In [1]:
# Setup: Add project root to path and change to examples directory
import sys
import os
from pathlib import Path

# Change to examples directory
os.chdir(Path.cwd() / 'examples' / 'getting_started' if 'getting_started' not in str(Path.cwd()) else Path.cwd())

# Add project root to path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"📁 Working directory: {Path.cwd()}")
print(f"📦 Project root: {project_root}")

📁 Working directory: c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi\examples\getting_started
📦 Project root: c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi


In [2]:
# Imports
import pandas as pd
import yaml

from odibi.pipeline import Pipeline
from odibi.config import PipelineConfig, ProjectConfig
from odibi.connections import LocalConnection
from odibi.registry import FunctionRegistry

# Import our transform functions
import transforms  # ← This registers all @transform functions

print("✅ All imports successful!")
print(f"📝 Registered transforms: {FunctionRegistry.list_functions()}")

✅ All imports successful!
📝 Registered transforms: ['calculate_revenue', 'filter_by_category', 'enrich_with_customer_data', 'aggregate_by_product']


d:\odibi\odibi\config.py:40: UserWarning: Field name "validate" in "BaseConnectionConfig" shadows an attribute in parent "BaseModel"
  class BaseConnectionConfig(BaseModel):
d:\odibi\odibi\config.py:156: UserWarning: Field name "validate" in "NodeConfig" shadows an attribute in parent "BaseModel"
  class NodeConfig(BaseModel):


---

## Step 1: Explore the Sample Data

Let's see what data we're working with.

In [3]:
# Load sample sales data
sales_df = pd.read_csv('data/sales.csv')

print("📊 Sales Data:")
print(f"  Rows: {len(sales_df)}")
print(f"  Columns: {list(sales_df.columns)}")
print("\nSample:")
sales_df.head()

📊 Sales Data:
  Rows: 10
  Columns: ['id', 'date', 'product', 'category', 'quantity', 'price', 'customer_id']

Sample:


,id,date,product,category,quantity,price,customer_id
0,1,2024-01-01,Widget A,Electronics,5,29.99,101
1,2,2024-01-01,Widget B,Electronics,3,49.99,102
2,3,2024-01-02,Gadget X,Home,2,15.50,103
3,4,2024-01-02,Widget A,Electronics,1,29.99,101
4,5,2024-01-03,Tool Y,Tools,4,12.00,104


In [4]:
# Load customer data
customers_df = pd.read_csv('data/customers.csv')

print("👥 Customer Data:")
print(f"  Rows: {len(customers_df)}")
print(f"  Columns: {list(customers_df.columns)}")
print("\nSample:")
customers_df.head()

👥 Customer Data:
  Rows: 7
  Columns: ['customer_id', 'name', 'region', 'tier']

Sample:


,customer_id,name,region,tier
0,101,Alice Smith,North,Gold
1,102,Bob Johnson,South,Silver
2,103,Carol White,East,Gold
3,104,David Brown,West,Bronze
4,105,Eve Davis,North,Silver


---

## Step 2: Simple Pipeline (Read → Write)

**Goal:** Load CSV, save as Parquet

**Pipeline:** `pipelines/simple.yaml`

In [5]:
# Load the simple pipeline config
with open('pipelines/simple.yaml') as f:
    pipeline_yaml = yaml.safe_load(f)

print("📋 Pipeline Config:")
print(yaml.dump(pipeline_yaml, default_flow_style=False))

📋 Pipeline Config:
description: Load CSV and save as Parquet
nodes:
- description: Load sales data from CSV
  name: load_sales
  read:
    connection: local_data
    format: csv
    path: sales.csv
- depends_on:
  - load_sales
  description: Save sales data as Parquet
  name: save_parquet
  write:
    connection: local_output
    format: parquet
    mode: overwrite
    path: sales.parquet
pipeline: simple_etl



In [6]:
# Create pipeline config from YAML
pipeline_config = PipelineConfig(**pipeline_yaml)

print(f"✅ Pipeline: {pipeline_config.pipeline}")
print(f"📦 Nodes: {[node.name for node in pipeline_config.nodes]}")

✅ Pipeline: simple_etl
📦 Nodes: ['load_sales', 'save_parquet']


In [8]:
# Setup connections
connections = {
    "local_data": LocalConnection(base_path="./data"),
    "local_output": LocalConnection(base_path="./output")
}

print("🔌 Connections configured:")
for name, conn in connections.items():
    print(f"  - {name}: {conn.base_path}")

🔌 Connections configured:
  - local_data: data
  - local_output: output


In [9]:
# Create and run pipeline
pipeline = Pipeline(pipeline_config, connections=connections)

# Validate first
validation = pipeline.validate()
print("🔍 Validation:")
print(f"  Valid: {validation['valid']}")
print(f"  Execution order: {validation['execution_order']}")

# Run pipeline
print("\n▶️ Running pipeline...\n")
results = pipeline.run()

print("\n✅ Pipeline Results:")
print(f"  Completed: {results.completed}")
print(f"  Failed: {results.failed}")
print(f"  Duration: {results.duration:.4f}s")

🔍 Validation:
  Valid: True
  Execution order: ['load_sales', 'save_parquet']

▶️ Running pipeline...


✅ Pipeline Results:
  Completed: ['load_sales', 'save_parquet']
  Failed: []
  Duration: 0.3377s


In [12]:
# Verify output file was created
import os

output_file = Path('output/sales.parquet')
if output_file.exists():
    print(f"✅ Output file created: {output_file}")
    print(f"   Size: {output_file.stat().st_size} bytes")
    
    # Load and verify
    saved_df = pd.read_parquet(output_file)
    print(f"   Rows: {len(saved_df)}")
    print("\n📊 First few rows:")
    display(saved_df.head())
else:
    print("❌ Output file not found!")

✅ Output file created: output\sales.parquet
   Size: 4601 bytes
   Rows: 10

📊 First few rows:


,id,date,product,category,quantity,price,customer_id
0,1,2024-01-01,Widget A,Electronics,5,29.99,101
1,2,2024-01-01,Widget B,Electronics,3,49.99,102
2,3,2024-01-02,Gadget X,Home,2,15.50,103
3,4,2024-01-02,Widget A,Electronics,1,29.99,101
4,5,2024-01-03,Tool Y,Tools,4,12.00,104


**🎉 Success!** You just ran your first ODIBI pipeline!

**What happened:**
1. Loaded `pipelines/simple.yaml`
2. Validated with Pydantic
3. Created Pipeline with connections
4. Executed: load_sales → save_parquet
5. Saved output to `output/sales.parquet`

---

## Step 3: Transform Pipeline (With Custom Functions)

**Goal:** Load data, calculate revenue, filter by category, save

**Pipeline:** `pipelines/transform.yaml`

**Uses:** Custom `@transform` functions from `transforms.py`

In [13]:
# Load transform pipeline
with open('pipelines/transform.yaml') as f:
    pipeline_yaml = yaml.safe_load(f)

print("📋 Transform Pipeline:")
print(f"  Pipeline: {pipeline_yaml['pipeline']}")
print(f"  Nodes: {[node['name'] for node in pipeline_yaml['nodes']]}")

📋 Transform Pipeline:
  Pipeline: transform_etl
  Nodes: ['load_sales', 'add_revenue', 'electronics_only', 'save_electronics']


In [14]:
# See the transform functions being used
for node in pipeline_yaml['nodes']:
    if 'transform' in node:
        print(f"\n🔧 {node['name']}:")
        for step in node['transform']['steps']:
            if isinstance(step, dict) and 'function' in step:
                print(f"  Function: {step['function']}")
                print(f"  Params: {step['params']}")


🔧 add_revenue:
  Function: calculate_revenue
  Params: {'source': 'load_sales'}

🔧 electronics_only:
  Function: filter_by_category
  Params: {'source': 'add_revenue', 'category': 'Electronics'}


In [15]:
# Create and run pipeline
pipeline_config = PipelineConfig(**pipeline_yaml)
pipeline = Pipeline(pipeline_config, connections=connections)

print("▶️ Running transform pipeline...\n")
results = pipeline.run()

print("\n✅ Results:")
print(f"  Completed: {results.completed}")
print(f"  Failed: {results.failed}")
print(f"  Duration: {results.duration:.4f}s")

▶️ Running transform pipeline...


✅ Results:
  Completed: ['load_sales', 'add_revenue', 'electronics_only', 'save_electronics']
  Failed: []
  Duration: 0.0461s


In [16]:
# Check the output
electronics_df = pd.read_csv('output/electronics_sales.csv')

print("📊 Electronics Sales Output:")
print(f"  Rows: {len(electronics_df)}")
print(f"  Columns: {list(electronics_df.columns)}")
print("\n💰 Revenue calculated and filtered:")
electronics_df

📊 Electronics Sales Output:
  Rows: 5
  Columns: ['id', 'date', 'product', 'category', 'quantity', 'price', 'customer_id', 'revenue']

💰 Revenue calculated and filtered:


,id,date,product,category,quantity,price,customer_id,revenue
0,1,2024-01-01,Widget A,Electronics,5,29.99,101,149.95
1,2,2024-01-01,Widget B,Electronics,3,49.99,102,149.97
2,4,2024-01-02,Widget A,Electronics,1,29.99,101,29.99
3,7,2024-01-04,Widget B,Electronics,2,49.99,102,99.98
4,9,2024-01-05,Widget A,Electronics,3,29.99,107,89.97


**What happened:**
1. Loaded sales data
2. **Added revenue column** using `calculate_revenue()` function
3. **Filtered for Electronics** using `filter_by_category()` function
4. Saved results

**Key insight:** Transform functions are just Python functions with `@transform` decorator!

---

## Step 4: Advanced Pipeline (SQL, Joins, Parallel)

**Goal:** Join sales with customers, aggregate by product

**Features:**
- Multiple data sources
- SQL transforms
- Function transforms
- Parallel writes

In [17]:
# Load advanced pipeline
with open('pipelines/advanced.yaml') as f:
    pipeline_yaml = yaml.safe_load(f)

pipeline_config = PipelineConfig(**pipeline_yaml)

print(f"📋 Pipeline: {pipeline_config.pipeline}")
print(f"📦 Nodes ({len(pipeline_config.nodes)}):")
for node in pipeline_config.nodes:
    deps = f" ← {node.depends_on}" if node.depends_on else ""
    print(f"  - {node.name}{deps}")

📋 Pipeline: advanced_etl
📦 Nodes (7):
  - load_sales
  - load_customers
  - sales_with_revenue ← ['load_sales']
  - enriched_sales ← ['sales_with_revenue', 'load_customers']
  - product_summary ← ['enriched_sales']
  - save_enriched ← ['enriched_sales']
  - save_summary ← ['product_summary']


In [ ]:
# Visualize the dependency graph
pipeline = Pipeline(pipeline_config, connections=connections)

print("🔀 Dependency Graph:\n")
print(pipeline.visualize())

In [ ]:
# See execution layers (what can run in parallel)
layers = pipeline.get_execution_layers()

print("⚡ Execution Layers (nodes in same layer can run in parallel):\n")
for i, layer in enumerate(layers, 1):
    print(f"  Layer {i}: {layer}")

In [ ]:
# Run the pipeline
print("▶️ Running advanced pipeline...\n")
results = pipeline.run()

print("\n✅ Results:")
print(f"  Completed: {results.completed}")
print(f"  Failed: {results.failed}")
print(f"  Duration: {results.duration:.4f}s")

print("\n📝 Node Details:")
for node_name in results.completed:
    node_result = results.get_node_result(node_name)
    print(f"  {node_name}: {node_result.duration:.4f}s")

In [ ]:
# Check enriched output
enriched_df = pd.read_csv('output/enriched_sales.csv')

print("📊 Enriched Sales (with customer data):")
print(f"  Rows: {len(enriched_df)}")
print(f"  Columns: {list(enriched_df.columns)}")
print("\nSample (notice customer name, region, tier):")
enriched_df.head()

In [ ]:
# Check product summary
summary_df = pd.read_csv('output/product_summary.csv')

print("📈 Product Summary (aggregated):")
summary_df

**🎉 Advanced pipeline complete!**

**What happened:**
1. Loaded **2 data sources** (sales + customers)
2. Used **SQL** to calculate revenue
3. **Joined** sales with customer data
4. **Aggregated** by product
5. Saved **2 outputs** (could run in parallel!)

**Notice:**
- `save_enriched` and `save_summary` are in Layer 4 (same layer)
- They could run in parallel (feature not implemented yet)
- Dependencies automatically handled!

---

## Step 5: Debugging - Run Single Nodes

**Problem:** You want to test one transform without running the whole pipeline.

**Solution:** `pipeline.run_node()` with mock data

In [ ]:
# Test the aggregate_by_product function in isolation

# Create mock data
mock_sales = pd.DataFrame({
    'product': ['Widget A', 'Widget A', 'Widget B', 'Widget B'],
    'quantity': [5, 3, 2, 4],
    'revenue': [100, 60, 80, 160]
})

print("🧪 Testing single node with mock data:\n")
print("Input:")
print(mock_sales)

# Run just the aggregate node
result = pipeline.run_node(
    "product_summary",
    mock_data={"enriched_sales": mock_sales}
)

print(f"\n✅ Node executed: {result.success}")
print(f"⏱ Duration: {result.duration:.4f}s")

print("\nOutput:")
pipeline.context.get("product_summary")

**Great for debugging!**
- Test transforms without loading real data
- Iterate quickly
- Debug failures in isolation

---

## Step 6: Error Handling

**What happens when things go wrong?**

In [ ]:
# Create a pipeline with an error
from odibi.config import NodeConfig, ReadConfig, TransformConfig

# Clear registry and add a failing function
FunctionRegistry._functions.clear()
FunctionRegistry._signatures.clear()

from odibi import transform

@transform
def failing_transform(context, source: str):
    """This will fail on purpose."""
    df = context.get(source)
    raise ValueError("Intentional failure for demo!")

# Create pipeline with error
error_pipeline_config = PipelineConfig(
    pipeline="error_demo",
    nodes=[
        NodeConfig(
            name="load",
            read=ReadConfig(connection="local_data", format="csv", path="sales.csv")
        ),
        NodeConfig(
            name="fail_node",
            depends_on=["load"],
            transform=TransformConfig(
                steps=[{"function": "failing_transform", "params": {"source": "load"}}]
            )
        ),
        NodeConfig(
            name="dependent",
            depends_on=["fail_node"],
            transform=TransformConfig(
                steps=["SELECT * FROM fail_node"]
            )
        ),
        NodeConfig(
            name="independent",
            read=ReadConfig(connection="local_data", format="csv", path="customers.csv")
        )
    ]
)

error_pipeline = Pipeline(error_pipeline_config, connections=connections)

print("▶️ Running pipeline with intentional error...\n")
results = error_pipeline.run()

print("\n📊 Results:")
print(f"  ✅ Completed: {results.completed}")
print(f"  ❌ Failed: {results.failed}")
print(f"  ⏭ Skipped: {results.skipped}")

In [ ]:
# Get error details
if results.failed:
    failed_node = results.failed[0]
    node_result = results.get_node_result(failed_node)
    
    print(f"❌ Node '{failed_node}' failed:\n")
    print(f"Error: {node_result.error}")

**Error Handling:**
- ✅ `load` completed (ran first)
- ❌ `fail_node` failed (intentional error)
- ⏭ `dependent` skipped (dependency failed)
- ✅ `independent` completed (no dependency on failed node)

**Philosophy:** 
- Don't fail fast
- Skip dependents of failed nodes
- Continue with independent nodes
- Collect all errors

---

## Step 7: Inspect the Context

**The Context** is how data passes between nodes.

In [ ]:
# Re-import transforms to register them again
import importlib
importlib.reload(transforms)

# Run the transform pipeline again
with open('pipelines/transform.yaml') as f:
    pipeline_yaml = yaml.safe_load(f)

pipeline_config = PipelineConfig(**pipeline_yaml)
pipeline = Pipeline(pipeline_config, connections=connections)
results = pipeline.run()

print("📦 After pipeline execution, context contains:\n")
print(f"  Registered DataFrames: {pipeline.context.list_names()}")

In [ ]:
# Access intermediate results from context
revenue_df = pipeline.context.get("add_revenue")

print("💰 Intermediate result 'add_revenue' (before filtering):")
print(f"  Rows: {len(revenue_df)}")
print(f"  Has revenue column: {'revenue' in revenue_df.columns}")
revenue_df.head()

In [ ]:
# Compare before and after filtering
electronics_df = pipeline.context.get("electronics_only")

print("📊 Before filtering (add_revenue):")
print(f"  Total rows: {len(revenue_df)}")
print(f"  Categories: {revenue_df['category'].unique().tolist()}")

print("\n📊 After filtering (electronics_only):")
print(f"  Total rows: {len(electronics_df)}")
print(f"  Categories: {electronics_df['category'].unique().tolist()}")
print(f"  Filtered out: {len(revenue_df) - len(electronics_df)} rows")

**Context is powerful!**
- Every node registers its result
- Downstream nodes access by name
- Great for debugging (inspect intermediate results)
- Same API for Spark and Pandas

---

## Step 8: Simplified Orchestration with PipelineManager

**Problem:** Manually loading YAML, creating configs, and setting up connections is verbose.

**Solution:** `PipelineManager` handles everything from a single project configuration file.

In [6]:
from odibi.pipeline import PipelineManager

# Initialize manager from project config (pipelines/manager_demo.yaml)
# This automatically:
# 1. Loads the project config
# 2. Sets up connections
# 3. Creates all pipelines
manager = PipelineManager.from_yaml("pipelines/manager_demo.yaml")

print(f"✅ Loaded pipelines: {manager.list_pipelines()}")

# Run the pipeline managed by the manager
print("\n▶️ Running managed pipeline...\n")
results = manager.run("simple_etl_managed")

print(f"\n✅ Success! Output saved to {results.story_path}")

✅ Loaded pipelines: ['simple_etl_managed']

▶️ Running managed pipeline...


Running pipeline: simple_etl_managed


✅ SUCCESS - simple_etl_managed
  Completed: 2 nodes
  Failed: 0 nodes
  Duration: 239.39s
  Story: c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi\examples\getting_started\output\stories\simple_etl_managed_20251119_122921.md

✅ Success! Output saved to c:\Users\hodibi\OneDrive - Ingredion\Desktop\Repos\Odibi\examples\getting_started\output\stories\simple_etl_managed_20251119_122921.md


**Why use PipelineManager?**
- Single source of truth (project config)
- Automatic connection setup
- Manages multiple pipelines
- Consistent execution environment

---

## Summary

**🎉 Congratulations!** You've learned:

### ✅ Core Concepts:
1. **YAML Configs** - Declarative pipeline definitions
2. **Nodes** - Read → Transform → Write units
3. **Dependencies** - Explicit `depends_on` declarations
4. **Context** - Data passing between nodes
5. **Connections** - Abstract data sources/destinations

### ✅ Features Used:
- ✅ CSV/Parquet read/write
- ✅ Transform functions (`@transform` decorator)
- ✅ SQL transforms (DuckDB)
- ✅ Multi-source joins
- ✅ Aggregation
- ✅ Dependency graph visualization
- ✅ Single node debugging
- ✅ Error handling

### 🚀 Next Steps:

**Try building your own pipeline:**
1. Create your own YAML config
2. Write custom `@transform` functions
3. Process your own data

**Explore more:**
- Read the framework docs: `docs/ODIBI_FRAMEWORK_PLAN.md`
- Review test examples: `test_exploration_phase2.ipynb`
- Check improvements list: `docs/IMPROVEMENTS.md`

**Happy data engineering!** 🎉